In [1]:
import pandas as pd
import numpy as np

In [2]:
x1 = ['AAA', 'BUY', 10, 10]
x2 = ['BBB', 'BUY', 12, 12]
x3 = ['CCC', 'BUY', 14, 14]
x33 = ['CCC','Cancel']
x4 = ['DDD', 'SELL', 15, 10]
x5 = ['EEE', 'SELL', 12, 2]

x6 = ['FFF', 'SELL', 12, 4]
x7 = ['GGG', 'SELL', 12, 10]
x77 = ['BBB','Cancel']
x8 = ['HHH', 'BUY', 12, 14]
x9 = ['KKK', 'SELL', 10, 20]
x99 = ['DDD','Cancel']
x10 = ['DDD','Cancel']

In [3]:
list2 = [x5, x6, x7, x8, x9]

In [4]:
listOfOrders = [x1, x2, x4]

In [5]:
def printInfoMatched(name, quantity, price, quantityLogic):
    if (quantityLogic):
            text = 'Fully' 
    else:
        text = 'Partially'
    print("{} matched with {} ({} @ {})".format(text, name, quantity, price))
        

In [314]:
def sell(df, name='EEE', side='SELL',price=12, quantity=12):
    try:
        index = df[(df.Side != side) & (df.Price >= price) & (df.Quantity >= quantity)].head(1).index[0]
        printInfoMatched(df.loc[index,'ID'], quantity, price, df.loc[index,'Quantity'] >= quantity)
        df.loc[index,'Quantity'] = df.loc[index,'Quantity'] - quantity
        if(quantity > df.loc[index,'Quantity']):
            sell(df, name, side, price, df.loc[index,'Quantity'] - quantity)
        else:
            index = df[(df.ID == name)].index[0]
            df = df.drop([index], inplace = True)
            
    except IndexError:
        elements = df[(df.Side != side) & (df.Price >= price) & (df.Quantity < quantity) & (df.Quantity!=0)].sort_values(by=['Price'],ascending=False).index
        for index in elements:  
            printInfoMatched(df.loc[index,'ID'], df.loc[index,'Quantity'], df.loc[index,'Price'], df.loc[index,'Quantity'] == quantity)
            if (df.loc[index,'Quantity'] - quantity <= 0):
                index2 = df[(df.ID == name)].index[0]
                df.loc[index2,'Quantity'] = quantity - df.loc[index,'Quantity']
                df.loc[index,'Quantity'] = 0
                quantity = df.loc[index2,'Quantity']

In [315]:
def buy(df, name='EEE', side='BUY',price=15, quantity=5):
    
    try:
        index = df[(df.Side != side) & (df.Price <= price) & (df.Quantity >= quantity)].head(1).index[0]
        printInfoMatched(df.loc[index,'ID'], df.loc[index,'Quantity'], df.loc[index,'Price'], df.loc[index,'Quantity'] == quantity)
        df.loc[index,'Quantity'] = df.loc[index,'Quantity'] - quantity
        if(quantity > df.loc[index,'Quantity']):
            buy(df, name, side, price, df.loc[index,'Quantity'] - quantity)
        else:
            index = df[(df.ID == name)].index[0]
            df = df.drop([index], inplace = True)
            
    except IndexError:
        #print("aaa")
        elements = df[(df.Side != side) & (df.Price <= price) & (df.Quantity < quantity) & (df.Quantity!=0)].sort_values(by=['Price'],ascending=False).index
        for index in elements:  
            printInfoMatched(df.loc[index,'ID'], df.loc[index,'Quantity'], df.loc[index,'Price'], df.loc[index,'Quantity'] <= quantity)
            if (df.loc[index,'Quantity'] - quantity <= 0):
                index2 = df[(df.ID == name)].index[0]
                df.loc[index2,'Quantity'] = quantity - df.loc[index,'Quantity']
                df.loc[index,'Quantity'] = 0
                quantity = df.loc[index2,'Quantity']
    

In [286]:
def cancel(df, name='EEE'):
    try:
        index = df[(df.ID == name)].index[0]
        if(df.loc[index , 'Quantity'] == 0):
            print("Failed - already fully filled")
        else:
            df.drop([index], inplace = True)
            print("OK")
    except:
        print("Failed – no such active order")
    

In [229]:
def addNew(df, name='EEE', side='SELL',price=12, quantity=2):
    #try:
    df = df.append({"ID":name,"Side":side,"Price":price,"Quantity":quantity}, ignore_index=True)
    return df
    #except:
    #    return pd.DataFrame({"ID":name,"Side":side,"Price":price,"Quantity":quantity})

In [230]:
def newOrder(dfOrginal, name, side, price, quantity):
    dfOrginal = addNew(dfOrginal, name, side, price, quantity)
    #print(dfOrginal)
    try:
        if(side=='SELL'):
            sell(dfOrginal, name, side, price, quantity)
        else:
            buy(dfOrginal, name, side, price, quantity)
    except:
        print("OK")
    #print(dfOrginal)
    return dfOrginal
    #print("Finish")

In [263]:
df2 = pd.DataFrame(listOfOrders, columns = ['ID', 'Side', 'Price', 'Quantity'])

In [264]:
df2

,ID,Side,Price,Quantity
0,AAA,BUY,10,10
1,BBB,BUY,12,12
2,DDD,SELL,15,10


In [316]:
%%time
df2 = pd.DataFrame(listOfOrders, columns = ['ID', 'Side', 'Price', 'Quantity'])
for i in list2:
    print('{} {} {} @ {}'.format(i[0], i[1], i[3], i[2]))
    df2 = newOrder(df2, i[0], i[1], i[2], i[3])
    print(df2)
    

EEE SELL 2 @ 12
Fully matched with BBB (2 @ 12)
    ID  Side  Price  Quantity
0  AAA   BUY     10        10
1  BBB   BUY     12        10
2  DDD  SELL     15        10
FFF SELL 4 @ 12
Fully matched with BBB (4 @ 12)
    ID  Side  Price  Quantity
0  AAA   BUY     10        10
1  BBB   BUY     12         6
2  DDD  SELL     15        10
GGG SELL 10 @ 12
Partially matched with BBB (6 @ 12)
    ID  Side  Price  Quantity
0  AAA   BUY     10        10
1  BBB   BUY     12         0
2  DDD  SELL     15        10
3  GGG  SELL     12         4
HHH BUY 14 @ 12
Fully matched with GGG (4 @ 12)
    ID  Side  Price  Quantity
0  AAA   BUY     10        10
1  BBB   BUY     12         0
2  DDD  SELL     15        10
3  GGG  SELL     12         0
4  HHH   BUY     12        10
KKK SELL 20 @ 10
Partially matched with HHH (10 @ 12)
Fully matched with AAA (10 @ 10)
    ID  Side  Price  Quantity
0  AAA   BUY     10         0
1  BBB   BUY     12         0
2  DDD  SELL     15        10
3  GGG  SELL     12       

In [284]:
df2

,ID,Side,Price,Quantity
0,AAA,BUY,10,0
1,BBB,BUY,12,0
2,DDD,SELL,15,0
3,GGG,SELL,12,0
4,HHH,BUY,12,0
5,KKK,SELL,10,10


In [157]:
df2 = newOrder(df2, x1[0], x1[1], x1[2], x1[3])

OK


In [158]:
df2
#.drop([0])

,ID,Side,Price,Quantity
0,0,0,0,0
1,BBB,BUY,12,12
2,AAA,BUY,10,10


In [152]:
df2 = pd.DataFrame([[0 , 0 , 0 , 0]], columns = ['ID', 'Side', 'Price', 'Quantity'])

In [156]:
df2 = newOrder(df2, x2[0], x2[1], x2[2], x2[3])

OK


In [127]:
df2 = newOrder(df2, x3[0], x3[1], x3[2], x3[3])

OK


In [133]:
cancel(df2, "CCC")

OK


In [135]:
df2 = newOrder(df2, x4[0], x4[1], x4[2], x4[3])

OK


In [137]:
df2 = newOrder(df2, x5[0], x5[1], x5[2], x5[3])

Partially matched with BBB (12 @ 12)


In [143]:
df2 = newOrder(df2, x6[0], x6[1], x6[2], x6[3])

Partially matched with BBB (4 @ 12)


In [145]:
df2 = newOrder(df2, x7[0], x7[1], x7[2], x7[3])

Partially matched with BBB (6 @ 12)


In [146]:
cancel(df2, "BBB")

Failed - already fully filled


In [148]:
df2 = newOrder(df2, x8[0], x8[1], x8[2], x8[3])

Partially matched with GGG (4 @ 12)


In [150]:
df2 = newOrder(df2, x9[0], x9[1], x9[2], x9[3])

Partially matched with AAA (10 @ 10)


In [279]:
df2[(df2.Side != 'SELL') & (df2.Price >= 0) & (df2.Quantity > 0)].sort_values(by=['Price'],ascending=False)

,ID,Side,Price,Quantity
1,BBB,BUY,12,12
0,AAA,BUY,10,10


In [277]:
df2

,ID,Side,Price,Quantity
0,AAA,BUY,10,10
1,BBB,BUY,12,12
2,DDD,SELL,15,10
